# Machine, Data & Learning

## Project | Genetic Algorithm

Team 30 | Tribrid
-----------
* Nitin Chandak (2019101024)
* Ayush Sharma (2019101004)

In [44]:
# Usefull Imports
import client as server
import numpy as np
import random
import json
%matplotlib inline

### Default Initial Overfit Vector
```
[
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]
```

In [45]:
# Usefull Global Constants & lambda functions

TEAM_ID = 'colthAUIKUTfdh4qWrnHBhJzkyEm8kt4qIue1BKtyvLItfp8Po'

DEFAULT_INITIAL_OVERFIT_VECTOR = [
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]

POPULATION_SIZE = 10
TOTAL_GENERATION = 0
MIN_GENE_VAL = -10
MAX_GENE_VAL = 10
MUTATION_PROBABILITY = 0.3
MUTATION_FACTOR = lambda : random.uniform(0.9, 1.1)
MATING_POOL_SIZE = lambda : random.randint(3,5)

In [46]:
# def create_default_output_file():
#     '''
#     This function will create a default
#     output file for dumping my 10 best vector
#     before any iteration of algorithm.
#     '''
#     answer = []
#     for i in range(10):
#         answer.append(DEFAULT_INITIAL_OVERFIT_VECTOR)
#     with open('output.txt','w') as write_file:
#         json.dump(answer, write_file)

In [47]:
# create_default_output_file()
# '''
# This function call has been made 
# only once to just make a starter 
# output.txt for our algorithm to work.
# '''

In [48]:
def read_output_file():
    '''
    This function will read the output.txt file
    and return the answer array there which contains
    10 best vector obtained so far.
    '''
    with open('output.txt','r') as write_file:
        answer=json.load(write_file)
    return answer